In [4]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
import os
import joblib
import numpy as np
import pandas as pd

In [5]:
ws = Workspace.from_config()
# choose a name for experiment
experiment_name = 'labor-prediction-experiment'
experiment=Experiment(ws, experiment_name)

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

datastore_name='workspaceblobstore'
datastore=Datastore.get(ws,datastore_name)

datastore_path = [(datastore, 'UI/01-03-2021_062128_UTC/LaborPredictionData6.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

ds = ds.take(3000).to_pandas_dataframe()

In [7]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8)

#y_train=y_train.reshape(1728, 1)
#train = np.concatenate((x_train,y_train), axis=1)
train=pd.concat([x_train, y_train], axis=1)

In [8]:
# TODO: Put your automl settings here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    training_data=train,
    label_column_name='TotalInstallationTime',
    n_cross_validations=10)

In [9]:
#TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_543a2f7e-3ccb-4f52-983f-3c94d1f64a02

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***************************************************

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
best_mlrun, fitted_mlmodel = remote_run.get_output()
print(best_mlrun)
print(fitted_mlmodel)
joblib.dump(fitted_mlmodel, "automlmodel.joblib")

Run(Experiment: labor-prediction-experiment,
Id: AutoML_543a2f7e-3ccb-4f52-983f-3c94d1f64a02_42,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             lo

['automlmodel.joblib']